# Parte Práctica 1 


Santiago Pereira

Nancy Mazariegos


In [7]:
!pip -q install simpy matplotlib >/dev/null 2>&1  

In [13]:
import random, statistics as stats
try:
    import simpy
except Exception as e:
    raise e

tiempos_totales = []
espera_registro = []
espera_consulta = []

def tiempo_registro(TIEMPO_REGISTRO_PROMEDIO): 
    import random
    return random.expovariate(1.0 / TIEMPO_REGISTRO_PROMEDIO)

def tiempo_consulta(TIEMPO_CONSULTA_PROMEDIO): 
    import random
    return random.expovariate(1.0 / TIEMPO_CONSULTA_PROMEDIO)

def interarribo(TASA_LLEGADA_PACIENTES): 
    import random
    return random.expovariate(1.0 / TASA_LLEGADA_PACIENTES)

def paciente(env, nombre, recepcionistas, medicos, TIEMPO_REGISTRO_PROMEDIO, TIEMPO_CONSULTA_PROMEDIO):
    llegada = env.now
    with recepcionistas.request() as req_r:
        t0 = env.now
        yield req_r
        espera_registro.append(env.now - t0)
        yield env.timeout(tiempo_registro(TIEMPO_REGISTRO_PROMEDIO))
    with medicos.request() as req_m:
        t1 = env.now
        yield req_m
        espera_consulta.append(env.now - t1)
        yield env.timeout(tiempo_consulta(TIEMPO_CONSULTA_PROMEDIO))
    tiempos_totales.append(env.now - llegada)

def generador_pacientes(env, recepcionistas, medicos, TASA_LLEGADA_PACIENTES, TIEMPO_REGISTRO_PROMEDIO, TIEMPO_CONSULTA_PROMEDIO):
    i = 0
    while True:
        yield env.timeout(interarribo(TASA_LLEGADA_PACIENTES))
        i += 1
        env.process(paciente(env, f"Paciente {i}", recepcionistas, medicos, TIEMPO_REGISTRO_PROMEDIO, TIEMPO_CONSULTA_PROMEDIO))

def reset_metricas():
    tiempos_totales.clear()
    espera_registro.clear()
    espera_consulta.clear()

def run_scenario(NUM_RECEPCIONISTAS=1, NUM_MEDICOS=2, TIEMPO_REGISTRO_PROMEDIO=2.0, TIEMPO_CONSULTA_PROMEDIO=7.0,
                 TASA_LLEGADA_PACIENTES=5.0, TIEMPO_SIMULACION=120.0, seed=42):
    if seed is not None:
        random.seed(seed)
    reset_metricas()
    env = simpy.Environment()
    recepcionistas = simpy.Resource(env, capacity=NUM_RECEPCIONISTAS)
    medicos = simpy.Resource(env, capacity=NUM_MEDICOS)
    env.process(generador_pacientes(env, recepcionistas, medicos, TASA_LLEGADA_PACIENTES, TIEMPO_REGISTRO_PROMEDIO, TIEMPO_CONSULTA_PROMEDIO))
    env.run(until=TIEMPO_SIMULACION)
    n = len(tiempos_totales)
    return {
        "pacientes_atendidos": n,
        "promedio_tiempo_total": (sum(tiempos_totales)/n) if n else float('nan'),
        "maximo_tiempo_total": max(tiempos_totales) if n else float('nan'),
        "promedio_espera_registro": (sum(espera_registro)/len(espera_registro)) if espera_registro else float('nan'),
        "promedio_espera_consulta": (sum(espera_consulta)/len(espera_consulta)) if espera_consulta else float('nan'),
        "params": {
            "NUM_RECEPCIONISTAS": NUM_RECEPCIONISTAS,
            "NUM_MEDICOS": NUM_MEDICOS,
            "TIEMPO_SIMULACION": TIEMPO_SIMULACION
        }
    }

## Escenarios

In [21]:
base = run_scenario(NUM_RECEPCIONISTAS=1, NUM_MEDICOS=2, TIEMPO_SIMULACION=120.0, seed=123)
base

{'pacientes_atendidos': 31,
 'promedio_tiempo_total': 16.006091723552643,
 'maximo_tiempo_total': 52.104213781751895,
 'promedio_espera_registro': 1.6341334184688272,
 'promedio_espera_consulta': 6.897476433130011,
 'params': {'NUM_RECEPCIONISTAS': 1,
  'NUM_MEDICOS': 2,
  'TIEMPO_SIMULACION': 120.0}}

In [22]:
med5 = run_scenario(NUM_RECEPCIONISTAS=1, NUM_MEDICOS=5, TIEMPO_SIMULACION=120.0, seed=123)
med5

{'pacientes_atendidos': 31,
 'promedio_tiempo_total': 8.388340293462068,
 'maximo_tiempo_total': 38.628105942392935,
 'promedio_espera_registro': 1.0773733613739864,
 'promedio_espera_consulta': 0.0,
 'params': {'NUM_RECEPCIONISTAS': 1,
  'NUM_MEDICOS': 5,
  'TIEMPO_SIMULACION': 120.0}}

In [23]:
rec2 = run_scenario(NUM_RECEPCIONISTAS=2, NUM_MEDICOS=2, TIEMPO_SIMULACION=120.0, seed=123)
rec2

{'pacientes_atendidos': 37,
 'promedio_tiempo_total': 10.47120914027494,
 'maximo_tiempo_total': 29.399043132734704,
 'promedio_espera_registro': 0.22451407427181477,
 'promedio_espera_consulta': 2.4139248654694003,
 'params': {'NUM_RECEPCIONISTAS': 2,
  'NUM_MEDICOS': 2,
  'TIEMPO_SIMULACION': 120.0}}

## Preguntas para el análisis

1. Configure la simulación con los parámetros iniciales (NUM_RECEPCIONISTAS = 1, NUM_MEDICOS = 2) y
proceda con su ejecución. Observe detenidamente la traza de eventos que su script imprime en la consola.

- ¿En qué etapa del proceso (registro o consulta) parece que los pacientes invierten la mayor parte de su
tiempo de espera? <u>Esperan más tiempo estando en una consulta.</u>

- ¿Cuál es el tiempo promedio total que un paciente permanece en la clínica según este
escenario base? <u>16 minutos.</u>

2. A continuación, usted deberá investigar cuál es el factor limitante principal del sistema. Mantenga el
número de recepcionistas en NUM_RECEPCIONISTAS = 1 y realice un cambio significativo en el número de
médicos, aumentándolo a NUM_MEDICOS = 5.

- a. Tras ejecutar la simulación, ¿observa usted una disminución sustancial en el tiempo promedio que
los pacientes pasan en la clínica? <u>Sí, y siento que es bastante grande ya que antes era un poco más de 16 minutos y bajó a 3.88, quien afectaba eran los médicos.</u>

- b. Con base en este resultado, ¿cuál de los dos recursos (recepcionistas o médicos) concluye usted
que es el cuello de botella del sistema? Por favor, explique su razonamiento. <u>Los médicos por que ellos eran los que estaban llenos pero aceleraban las consultas</u>


3. Restablezca el número de médicos a su valor original (NUM_MEDICOS = 2). Ahora, dirija su atención al
recurso que identificó como el cuello de botella en la pregunta anterior. Si fue el recepcionista, modifique el
parámetro a NUM_RECEPCIONISTAS = 2 y ejecute la simulación.

- a. Compare el tiempo promedio resultante en la clínica de este nuevo escenario con el obtenido en la
línea base (pregunta 1). <u>Si disminuypo pero no tanro como en la pregunta anterior ya que había dado 16 minutos y ahora dio 10.47 minutos.</u>

- b. ¿Qué intervención resultó más efectiva para mejorar el rendimiento global del sistema: el
incremento de médicos (pregunta 2) o el de recepcionistas? <u>Tener más médicos que recepcionistas.</u>

4. Asuma el rol de un consultor de procesos contratado para optimizar la operación de esta clínica.
Basándose en la evidencia cuantitativa recopilada de sus tres experimentos (línea base, incremento de
médicos, incremento de recepcionistas), redacte una recomendación profesional y concisa para la
dirección de la clínica. ¿En qué recurso debería invertir la clínica para lograr la reducción más significativa
en los tiempos de espera, considerando el menor costo posible (bajo el supuesto de que el costo de un
recepcionista es inferior al de un médico)? <u>Con los tres experimentos queda claro que el atasco está en consulta: al sumar médicos el tiempo total bajó mucho más que al sumar un recepcionista Mi recomendación, en simple: inviertan primero en más médicos (aunque sea por turnos u horas pico) porque ahí se elimina la cola grande y la espera cae fuerte; si el presupuesto es corto y, como asumimos, un recepcionista cuesta menos, agregar uno puede dar un alivio rápido y barato pero no quita la fila de consulta. En resumen: para el mayor impacto, amplíen la capacidad médica,  el recepcionista extra es útil como apoyo, no como solución principal.</u>
